# STC Jawwy

In [4]:
"""
Here we install libraries that are not installed by default
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
# !pip install pyxlsb # Not needed for .xlsx files

'\nHere we install libraries that are not installed by default\nExample:  pyslsb\nFeel free to add any library you are planning to use.\n'

In [5]:
# Import the required libraries
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
# import pyxlsb           # Excel extention to read xlsb files (the input file) # Not needed for .xlsx files
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset includes total watching hours for customers per day.

You are required to work on predecting the forecast for the watching hours.

In [6]:
dataframe = pd.read_excel("/content/stc TV Data Set_T2.xlsx", index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [7]:
# check the data shape
dataframe.shape

(86, 2)

In [8]:
# display the first 5 rows
dataframe.head()

,date_,Total_watch_time_in_houres
0,2018-01-01,1123.551944
1,2018-01-02,1000.129722
2,2018-01-03,881.924444
3,2018-01-04,782.669444
4,2018-01-05,1051.939444


In [9]:
# display the dataset after applying data types
dataframe.head()

,date_,Total_watch_time_in_houres
0,2018-01-01,1123.551944
1,2018-01-02,1000.129722
2,2018-01-03,881.924444
3,2018-01-04,782.669444
4,2018-01-05,1051.939444


In [10]:
# describe the numeric values in the dataset
dataframe.describe()

,date_,Total_watch_time_in_houres
count,86,86.000000
mean,2018-02-28 17:01:23.720930304,780.817926
min,2018-01-01 00:00:00,562.124722
25%,2018-01-30 06:00:00,707.709653
50%,2018-02-28 12:00:00,763.181389
75%,2018-03-29 18:00:00,840.985278
max,2018-04-30 00:00:00,1123.551944
std,NaN,122.992002


In [11]:
# check if any column has null value in the dataset
dataframe.isnull().any()

,0
date_,False
Total_watch_time_in_houres,False


In [12]:
# we import Visualization libraries
# you can ignore and use any other graphing libraries
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [13]:
# Setting the date as index
dataframe.set_index('date_', inplace=True)

In [14]:
# Display the dataframe after setting the date as index
dataframe.head()

,Total_watch_time_in_houres
date_,
2018-01-01,1123.551944
2018-01-02,1000.129722
2018-01-03,881.924444
2018-01-04,782.669444
2018-01-05,1051.939444


In [15]:
# show the dataframe
fig = px.line(dataframe,  y="Total_watch_time_in_houres")
fig.show()

In [37]:
"""
TODO using the previous dataset (df) build a prediction model to predict the expected watch time for the next two months
Hint: you can build a forecast model to predict the results
"""

# Import additional libraries for forecasting
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')


In [38]:
# Create a working copy of the dataframe
df = dataframe.copy()

# Data preprocessing
if 'date_' in df.columns:
    df['date_'] = pd.to_datetime(df['date_'])
    df.set_index('date_', inplace=True)
else:
    df.index = pd.to_datetime(df.index)

In [39]:
# Create features for prediction
df['day_of_week'] = df.index.dayofweek
df['day_of_month'] = df.index.day
df['month'] = df.index.month
df['rolling_7_avg'] = df['Total_watch_time_in_houres'].rolling(window=7).mean()
df['lag_1'] = df['Total_watch_time_in_houres'].shift(1)
df['time_index'] = range(len(df))

# Remove rows with NaN values
df_clean = df.dropna()

In [40]:
# Prepare training data
feature_columns = ['day_of_week', 'day_of_month', 'month', 'rolling_7_avg', 'lag_1', 'time_index']
train_size = int(len(df_clean) * 0.8)

X_train = df_clean[feature_columns][:train_size]
y_train = df_clean['Total_watch_time_in_houres'][:train_size]
X_test = df_clean[feature_columns][train_size:]
y_test = df_clean['Total_watch_time_in_houres'][train_size:]

In [41]:
# Train models
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Select best model
linear_pred = linear_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

linear_r2 = r2_score(y_test, linear_pred)
rf_r2 = r2_score(y_test, rf_pred)

if linear_r2 > rf_r2:
    best_model = linear_model
    model_name = "Linear Regression"
    best_r2 = linear_r2
else:
    best_model = rf_model
    model_name = "Random Forest"
    best_r2 = rf_r2

print(f"Best Model: {model_name} (R² = {best_r2:.3f})")

Best Model: Linear Regression (R² = 0.002)


In [42]:
# Generate predictions for next 60 days
last_date = df_clean.index[-1]
future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=60, freq='D')

In [43]:
# Create future features
future_df = pd.DataFrame(index=future_dates)
future_df['day_of_week'] = future_df.index.dayofweek
future_df['day_of_month'] = future_df.index.day
future_df['month'] = future_df.index.month
future_df['rolling_7_avg'] = df_clean['rolling_7_avg'].iloc[-1]
future_df['lag_1'] = df_clean['Total_watch_time_in_houres'].iloc[-1]
future_df['time_index'] = range(len(df_clean), len(df_clean) + 60)

# Make predictions
future_predictions = best_model.predict(future_df[feature_columns])
future_df['predicted_watch_time'] = future_predictions

In [44]:
# Peak hours analysis
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_analysis = df_clean.groupby('day_of_week')['Total_watch_time_in_houres'].mean()
actual_day_names = [day_names[i] for i in weekly_analysis.index]
weekly_analysis.index = actual_day_names

# Identify peak days (top 25%)
peak_threshold = np.percentile(future_predictions, 75)
peak_days = future_df[future_df['predicted_watch_time'] >= peak_threshold]

print(f"\nNext 2 Months Prediction Summary:")
print(f"Average predicted watch time: {future_predictions.mean():.2f} hours/day")
print(f"Peak days (top 25%): {len(peak_days)} days")
print(f"Peak threshold: {peak_threshold:.2f} hours")

print(f"\nWeekly Pattern:")
for day, avg_hours in weekly_analysis.items():
    print(f"{day}: {avg_hours:.2f} hours")


Next 2 Months Prediction Summary:
Average predicted watch time: 630.54 hours/day
Peak days (top 25%): 15 days
Peak threshold: 676.90 hours

Weekly Pattern:
Monday: 766.15 hours
Tuesday: 771.80 hours
Wednesday: 759.05 hours
Thursday: 767.01 hours
Friday: 777.80 hours


In [45]:
# Top 5 peak days
print(f"\nTop 5 Predicted Peak Days:")
top_days = future_df.nlargest(5, 'predicted_watch_time')
for date, row in top_days.iterrows():
    day_name = day_names[int(row['day_of_week'])]
    print(f"{date.strftime('%Y-%m-%d')} ({day_name}): {row['predicted_watch_time']:.2f} hours")



Top 5 Predicted Peak Days:
2018-05-06 (Sunday): 738.71 hours
2018-05-05 (Saturday): 738.33 hours
2018-05-04 (Friday): 737.96 hours
2018-05-03 (Thursday): 737.59 hours
2018-05-02 (Wednesday): 737.21 hours


In [46]:
# Simple visualization
fig = make_subplots(rows=1, cols=2, subplot_titles=('Prediction vs Historical Data', 'Weekly Patterns'))

# Historical and predicted data
fig.add_trace(go.Scatter(x=df_clean.index, y=df_clean['Total_watch_time_in_houres'],
                        mode='lines', name='Historical', line=dict(color='blue')),
              row=1, col=1)
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['predicted_watch_time'],
                        mode='lines', name='Predicted', line=dict(color='red')),
              row=1, col=1)

# Weekly patterns
fig.add_trace(go.Bar(x=actual_day_names, y=weekly_analysis.values,
                    name='Weekly Average', marker_color='green'),
              row=1, col=2)

fig.update_layout(height=400, title_text="Watch Time Analysis and Forecast")
fig.show()


In [47]:
# Save results
output_df = pd.DataFrame({
    'date': future_df.index,
    'predicted_hours': future_predictions,
    'day_of_week': [day_names[int(dow)] for dow in future_df['day_of_week']],
    'is_peak': future_predictions >= peak_threshold
})

output_df.to_csv('prediction_results.csv', index=False)
print(f"\nResults saved to: prediction_results.csv")


Results saved to: prediction_results.csv
